# Channel Prediction Model for Quality Control

This notebook contains the code and outlines the development of the
SSMI Channel Prediction Neural Network for QC purposes of SSMI L1C
data. The simple Feed-Forward Neural Networks (FFNNs) are trained on
good quality SSMIS brightness temperature observation vectors and are
trained to predict one channel from all the others as predictors.

The end product is a tree of FFNNs, discretized by surface (ocean or 
land), and by channel, since these are fully distinct problems.

In [3]:


import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import glob
import torch
from torch import nn
import cartopy.crs as ccrs
from util_funcs.L1C import scantime2datetime
from util_funcs import data2xarray, array_funcs
import geography
from tqdm import tqdm

from dataset_class import dataset
from model_class import channel_predictor


#General parameters:

satellite = 'F13'
sensor = 'SSMI'

nchans = 5
batch_size = 1000
input_size = nchans - 1
hidden_size = 256
output_size = 1

In [4]:
'''
Function Definitions:
'''


# Tb array will be set up as follows:
#     Tbs =  [m x n], where m is the number of samples and n is the
#            number of channels (features)
#     1-2:   19.35 V and H
#     3:     22.235 V
#     4-5:   37.0 V and H
#     6-9:   85.5 V and H #Double-sampled 85

chan_desc = np.array(['19V', '19H', '22V', '37V', '37H'])


def extract_channel(Tb_array, chan):

    '''
    Use in preparing training data.

    Assumes Tb array is [m x n] where m (rows) are samples
    and n (columns) is the number of channels.

    Passing in the channel description splits the data so
    that the specified channel is its own vector y and the
    rest are kept as predictors x.

    Inputs:
        Tb_array    |  ndarray of Tbs
        chan        |  string of channel name
    Outputs:
        x           |  matrix of predictors (other channels)
        y           |  vector of predictands (the missing channel)
        
    '''
    
    chan_desc = np.array(['19V', '19H', '22V', '37V', '37H'])


    chan_indx = np.where(chan == chan_desc)[0]

    if np.size(chan_indx) == 0:
        raise ValueError(f'Channel description must be in list {chan_desc}.')

    y = Tb_array[:,chan_indx]
    x = Tb_array[:,np.delete(np.arange(0,len(chan_desc)),chan_indx)]

    return x, y



def split_data_indcs(x, train=80, test=10, val=10, device=None, randomize=False):

    '''
    Creates train/test/validation split for training data. Default is 80%/10%/10%.

    Inputs:
        x       |   Array of predictors. Expects shape (nsamples, nfeatures).
        y       |   Array of predictands. Expects shape (nsamples, nfeatures).
        train   |   Percentage of data to be used for training.
        test    |   Percentage of data to be used for testing.
        val     |   Percentage of data to be used for validation.
                        -Train + test + val must equal 100.
        device (optional)    |  Either 'cuda' or 'cpu'. 
        randomize (optional) |  Whether to shuffle the data before creating
                                    the splits. This functionality is currently
                                    not supported but is intended to be 
                                    implemented in the future.
    '''

    if train + test + val != 100:
        raise ValueError(f'train + test + val must equal 100%.')

    # if x.shape[0] != y.shape[0]:
    #     raise ValueError(f'Dimensions of x {x.shape} and y {y.shape} not compatible.')

    nsamples = x.shape[0]

    ntrain = int(nsamples * (train / 100.))
    ntest  = int(nsamples * (test / 100.))
    nval   = nsamples - ntrain - ntest

    indcs = np.arange(0,nsamples)

    train_indcs = indcs[0:ntrain]
    test_indcs  = indcs[ntrain:ntrain+ntest]
    val_indcs   = indcs[ntrain+ntest:]

    return train_indcs, test_indcs, val_indcs


def train_model(model, nepochs, dataloader, learning_rate=0.001, quiet=False, stage=None, validation_dataloader=None):

    nbatches = len(dataloader)
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=nepochs)
    device = None

    if stage:
        print(f'Training stage: {stage}')

    loss_arr    = np.zeros([nbatches,nepochs], dtype='f')
    valloss_arr = np.zeros([nbatches,nepochs], dtype='f')
    
    for epoch in range(nepochs):
        for i, (profs, obs) in enumerate(dataloader):
            if device:
                profs, obs = profs.to(device), obs.to(device)

            if validation_dataloader and i%100==0:
                valprofs, valobs = next(enumerate(validation_dataloader))[1]
                val_pred = model(valprofs)
                valloss  = criterion(val_pred, valobs)
                print(f'Validation Loss = {valloss.item():.3f}')

            #Forward pass:
            obs_pred = model(profs)
            loss     = criterion(obs_pred, obs)


            #Backward pass:
            optimizer.zero_grad()
            loss.backward()

            #Update neurons:
            optimizer.step()

            loss_arr[i,epoch] = loss.item()
            valloss_arr[i,epoch] = valloss.item()
            
            if not quiet:
                if i%100 == 0:
                    print(f'Epoch = {epoch+1}, batch = {i} of {nbatches}, loss = {loss.item():.3f}, LR = {scheduler.get_last_lr()[0]}')
        
        scheduler.step()




    return loss_arr, valloss_arr



In [8]:
'''

SSMI CHANNEL PREDICTION MODEL:
    1: Ocean

'''

sfc = [1]

with xr.open_dataset(f'training_data/{satellite}_training_data.nc') as f:
    
    sfctype = f.sfctype.values

    correct_sfc = np.isin(sfctype, sfc)
    sfcindcs = np.where(correct_sfc)[0]

    Tbs = f.Tbs.values[sfcindcs]

#---Don't shuffle data before splitting
#np.random.seed(40)
#Tbs = array_funcs.shuffle_data(Tbs, axis=0)

#Get rid of 89s:
Tbs = Tbs[:,:-4]

if Tbs.shape[0] > 5.0e+06:
    Tbs = Tbs[:5_000_000,:]


print(Tbs.shape)

#---Split data into train/test/val:
train_indcs, test_indcs, val_indcs = split_data_indcs(Tbs)

Tbs_train = Tbs[train_indcs]
Tbs_test  = Tbs[test_indcs]
Tbs_val   = Tbs[val_indcs]

#---Shuffle before converting to tensors:
np.random.seed(40)
Tbs_train = array_funcs.shuffle_data(Tbs_train, axis=0)


(5000000, 5)


In [ ]:
'''
Predict channels: Train all models
'''

torch.set_num_threads(10)

for ichan, channel in enumerate(chan_desc):

    #---Extract channel, x = predictors, y = channel to predict
    x_train, y_train = extract_channel(Tbs_train, channel)
    x_test,  y_test  = extract_channel(Tbs_test, channel)
    x_val,   y_val   = extract_channel(Tbs_val, channel)
    
    x_train, y_train = torch.tensor(x_train), torch.tensor(y_train)
    x_test,  y_test  = torch.tensor(x_test), torch.tensor(y_test)
    x_val,   y_val   = torch.tensor(x_val), torch.tensor(y_val)


    #---Set up dataloaders:
    train_loader = torch.utils.data.DataLoader(dataset=dataset(x_train,y_train), 
                                               batch_size=batch_size, 
                                               shuffle=True, drop_last=True)
    test_loader = torch.utils.data.DataLoader(dataset=dataset(x_test,y_test), 
                                               batch_size=None, 
                                               shuffle=False, drop_last=False)
    val_loader = torch.utils.data.DataLoader(dataset=dataset(x_val,y_val), 
                                               batch_size=None, 
                                               shuffle=False, drop_last=False)

    #---Create model:
    model = channel_predictor(input_size, hidden_size, output_size)

    #---Train_model:
    nbatches = len(train_loader)
    nepochs_stage1 = 5
    nepochs_stage2 = 10
    nepochs_stage3 = 20

    loss_stage1, valloss_stage1 = train_model(model, nepochs=5, dataloader=train_loader, 
                                          learning_rate=0.001, quiet=False, stage=1, validation_dataloader=val_loader)
    loss_stage2, valloss_stage2 = train_model(model, nepochs=10, dataloader=train_loader, 
                                          learning_rate=0.001, quiet=False, stage=2, validation_dataloader=val_loader)
    loss_stage3, valloss_stage3 = train_model(model, nepochs=20, dataloader=train_loader, 
                                          learning_rate=0.001, quiet=False, stage=3, validation_dataloader=val_loader)

    torch.save(model.state_dict(), f'models/{sensor}_{satellite}_channel_predictor_{channel}_ocean.pt')

    loss_data = data2xarray(data_vars = (loss_stage1, loss_stage2, loss_stage3, 
                                     valloss_stage1, valloss_stage2, valloss_stage3),
                        var_names = ('LossStage1','LossStage2','LossStage3',
                                     'ValidationLossStage1', 'ValidationLossStage2', 'ValidationLossStage3'),
                        dims = (nbatches,nepochs_stage1,nepochs_stage2, nepochs_stage3),
                        dim_names = ('training_batches', 'epochs_stage1', 'epochs_stage2', 'epochs_stage3'))

    loss_data.to_netcdf(f'diagnostics/loss_data_{channel}_ocean.nc', engine='netcdf4')

    print(f'Finished training model for channel {channel}.')

Training stage: 1
Validation Loss = 39554.375
Epoch = 1, batch = 0 of 4000, loss = 38317.957, LR = 0.001
Validation Loss = 54.737
Epoch = 1, batch = 100 of 4000, loss = 14.577, LR = 0.001
Validation Loss = 10.782
Epoch = 1, batch = 200 of 4000, loss = 2.653, LR = 0.001
Validation Loss = 3.881
Epoch = 1, batch = 300 of 4000, loss = 2.297, LR = 0.001
Validation Loss = 2.424
Epoch = 1, batch = 400 of 4000, loss = 2.015, LR = 0.001
Validation Loss = 2.438
Epoch = 1, batch = 500 of 4000, loss = 1.870, LR = 0.001
Validation Loss = 2.352
Epoch = 1, batch = 600 of 4000, loss = 1.927, LR = 0.001
Validation Loss = 1.801
Epoch = 1, batch = 700 of 4000, loss = 1.795, LR = 0.001
Validation Loss = 1.758
Epoch = 1, batch = 800 of 4000, loss = 1.593, LR = 0.001
Validation Loss = 1.091
Epoch = 1, batch = 900 of 4000, loss = 1.824, LR = 0.001
Validation Loss = 2.027
Epoch = 1, batch = 1000 of 4000, loss = 1.460, LR = 0.001
Validation Loss = 1.658
Epoch = 1, batch = 1100 of 4000, loss = 1.461, LR = 0.001

In [10]:
#General parameters:
nchans = 5
batch_size = 1000
input_size = nchans - 1 + 1 #Remaining channels + surface type
hidden_size = 256
output_size = 1

In [11]:
'''

SSMI CHANNEL PREDICTION MODEL:
    2: Non-Ocean Surfaces

'''

with xr.open_dataset(f'training_data/{satellite}_training_data.nc') as f:
    
    sfctype = f.sfctype.values

    correct_sfc = sfctype > 1

    Tbs = f.Tbs.values[correct_sfc,:]
    sfctype = sfctype[correct_sfc]


#Get rid of 89s:
Tbs = Tbs[:,:-4]

#---Split data into train/test/val:
train_indcs, test_indcs, val_indcs = split_data_indcs(Tbs)

Tbs_train = Tbs[train_indcs]
Tbs_test  = Tbs[test_indcs]
Tbs_val   = Tbs[val_indcs]

sfctype_train = sfctype[train_indcs].astype(np.float32)
sfctype_test  = sfctype[test_indcs].astype(np.float32)
sfctype_val   = sfctype[val_indcs].astype(np.float32)

#---Shuffle before converting to tensors:
np.random.seed(40)
Tbs_train, shuffled_indcs = array_funcs.shuffle_data(Tbs_train, axis=0, return_indcs=True)
sfctype_train = sfctype_train[shuffled_indcs]

In [12]:
sfctype_train, sfctype_test, sfctype_val

(array([ 2.,  5., 13., ...,  2., 11., 15.], shape=(7748290,), dtype=float32),
 array([ 8.,  8.,  8., ..., 11., 11., 11.], shape=(968536,), dtype=float32),
 array([11., 11., 11., ..., 11., 11., 11.], shape=(968537,), dtype=float32))

In [ ]:
'''
Predict channels: Train all models
'''

torch.set_num_threads(10)

for ichan, channel in enumerate(chan_desc):

    #---Extract channel, x = predictors, y = channel to predict
    x_train, y_train = extract_channel(Tbs_train, channel)
    x_test,  y_test  = extract_channel(Tbs_test, channel)
    x_val,   y_val   = extract_channel(Tbs_val, channel)

    x_train = np.concatenate((x_train, sfctype_train[:,None]), axis=1)
    x_test  = np.concatenate((x_test,  sfctype_test[:,None]), axis=1)
    x_val   = np.concatenate((x_val,   sfctype_val[:,None]), axis=1)
    
    x_train, y_train = torch.tensor(x_train), torch.tensor(y_train)
    x_test,  y_test  = torch.tensor(x_test), torch.tensor(y_test)
    x_val,   y_val   = torch.tensor(x_val), torch.tensor(y_val)


    #---Set up dataloaders:
    train_loader = torch.utils.data.DataLoader(dataset=dataset(x_train,y_train), 
                                               batch_size=batch_size, 
                                               shuffle=True, drop_last=True)
    test_loader = torch.utils.data.DataLoader(dataset=dataset(x_test,y_test), 
                                               batch_size=None, 
                                               shuffle=False, drop_last=False)
    val_loader = torch.utils.data.DataLoader(dataset=dataset(x_val,y_val), 
                                               batch_size=None, 
                                               shuffle=False, drop_last=False)

    #---Create model:
    model = channel_predictor(input_size, hidden_size, output_size)

    #---Train_model:
    nbatches = len(train_loader)
    nepochs_stage1 = 5
    nepochs_stage2 = 10
    nepochs_stage3 = 20

    loss_stage1, valloss_stage1 = train_model(model, nepochs=5, dataloader=train_loader, 
                                          learning_rate=0.001, quiet=False, stage=1, validation_dataloader=val_loader)
    loss_stage2, valloss_stage2 = train_model(model, nepochs=10, dataloader=train_loader, 
                                          learning_rate=0.001, quiet=False, stage=2, validation_dataloader=val_loader)
    loss_stage3, valloss_stage3 = train_model(model, nepochs=20, dataloader=train_loader, 
                                          learning_rate=0.001, quiet=False, stage=3, validation_dataloader=val_loader)

    torch.save(model.state_dict(), f'models/{sensor}_{satellite}_channel_predictor_{channel}_nonocean.pt')

    loss_data = data2xarray(data_vars = (loss_stage1, loss_stage2, loss_stage3, 
                                     valloss_stage1, valloss_stage2, valloss_stage3),
                        var_names = ('LossStage1','LossStage2','LossStage3',
                                     'ValidationLossStage1', 'ValidationLossStage2', 'ValidationLossStage3'),
                        dims = (nbatches,nepochs_stage1,nepochs_stage2, nepochs_stage3),
                        dim_names = ('training_batches', 'epochs_stage1', 'epochs_stage2', 'epochs_stage3'))

    loss_data.to_netcdf(f'diagnostics/loss_data_{channel}_nonocean.nc', engine='netcdf4')

    print(f'Finished training model for channel {channel}.')

Training stage: 1
Validation Loss = 41041.945
Epoch = 1, batch = 0 of 7748, loss = 51357.312, LR = 0.001
Validation Loss = 6.805
Epoch = 1, batch = 100 of 7748, loss = 38.479, LR = 0.001
Validation Loss = 4.460
Epoch = 1, batch = 200 of 7748, loss = 17.008, LR = 0.001
Validation Loss = 5.430
Epoch = 1, batch = 300 of 7748, loss = 18.186, LR = 0.001
Validation Loss = 6.189
Epoch = 1, batch = 400 of 7748, loss = 15.277, LR = 0.001
Validation Loss = 4.049
Epoch = 1, batch = 500 of 7748, loss = 16.288, LR = 0.001
Validation Loss = 0.076
Epoch = 1, batch = 600 of 7748, loss = 11.692, LR = 0.001
Validation Loss = 0.002
Epoch = 1, batch = 700 of 7748, loss = 11.210, LR = 0.001
Validation Loss = 0.009
Epoch = 1, batch = 800 of 7748, loss = 8.332, LR = 0.001
Validation Loss = 0.064
Epoch = 1, batch = 900 of 7748, loss = 9.273, LR = 0.001
Validation Loss = 0.084
Epoch = 1, batch = 1000 of 7748, loss = 8.091, LR = 0.001
Validation Loss = 0.023
Epoch = 1, batch = 1100 of 7748, loss = 7.288, LR = 0